<a class="anchor" id="0"></a>
# Air Quality City - for Vinnytsia city - Data Mapping

# Datasets:
* [Air Quality Monitoring from EcoCity](https://www.kaggle.com/vbmokin/air-quality-monitoring-from-ecocity)
* [Air Quality Monitoring](https://www.kaggle.com/vbmokin/air-quality-monitoring)

## Acknowledgements

### Notebooks:
* [Air Quality City - Prediction & Mapping](https://www.kaggle.com/code/vbmokin/air-quality-city-prediction-mapping)
* [Air Quality in City - 2D Analysis](https://www.kaggle.com/vbmokin/air-quality-in-city-2d-analysis)
* [Air Quality in Region - 2D Analysis](https://www.kaggle.com/vbmokin/air-quality-in-region-2d-analysis)
* [Data Science for tabular data: Advanced Techniques](https://www.kaggle.com/vbmokin/data-science-for-tabular-data-advanced-techniques)
* [EDA for tabular data: Advanced Techniques](https://www.kaggle.com/vbmokin/eda-for-tabular-data-advanced-techniques)

### Kaggle Datasets:
* [Air Quality Monitoring from EcoCity](https://www.kaggle.com/vbmokin/air-quality-monitoring-from-ecocity)
* [Air Quality Monitoring](https://www.kaggle.com/vbmokin/air-quality-monitoring)

### Thanks to: 
* https://pypi.org/project/gmplot/
* https://livecodestream.dev/post/how-to-plot-your-data-on-maps-using-python-and-folium/

<a class="anchor" id="0.1"></a>
## Table of Contents

1. [Import libraries](#1)
1. [Download data](#2)
   - [Download data from SaveEcoBot](#2.1)
   - [Download data from EcoCity](#2.2)
   - [Data aggregation](#2.3)
1. [Prediction](#3)
1. [Mapping](#4)

## 1. Import libraries<a class="anchor" id="1"></a>

[Back to Table of Contents](#0.1)

In [1]:
# Work with Data - the main Python libraries
import numpy as np
import pandas as pd
from scipy.interpolate import interp2d
import datetime

# For import data
import os
import json
import requests

# Visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import folium

# Modeling and Prediction
from sklearn.metrics import r2_score

import warnings
warnings.simplefilter('ignore')

In [2]:
indicator_name = 'PM2.5' # 'PM2.5' or 'PM10'
time_interval='H' # 'H' (hour) or D' (day)
type_agg='mean' # 'mean' or 'max'

In [3]:
#datetime_analysis = '2021-11-16 10:00:00'
datetime_analysis = '2021-11-27 09:00:00'  # maximum value after 2021-11-16
#datetime_analysis = '2021-11-12 18:00:00'
#datetime_analysis = '2021-01-23 18:00:00'  # maximum value

## 2. Download data<a class="anchor" id="2"></a>

[Back to Table of Contents](#0.1)

### 2.1 Download data from SaveEcoBot<a class="anchor" id="2.1"></a>

[Back to Table of Contents](#0.1)

In [4]:
# Data from SaveEcoBot
stations_about = pd.read_csv('../input/air-quality-monitoring/saveecobot_city_about_stations.csv', header=0, sep=';')
stations_about = stations_about[stations_about['locality']=='Vinnytsia city'].reset_index(drop=True)
stations_about = stations_about.fillna(-1)
stations_about

,id_saveecobot,id_ecocity,network,locality,address,start_date,notes,lat,lng
0,14627,848.0,Eco-City,Vinnytsia city,"2nd Konstantynovycha Ln, 4",14.12.2020,-1,49.2307,28.4371
1,4152,767.0,Eco-City,Vinnytsia city,"Soborna St, 36",26.06.2020,Vinnytsia City Concil,49.2329,28.4704
2,1008,-1.0,SaveDnipro,Vinnytsia city,"Stetsenka St, 75",25.05.2019,-1,49.2393,28.4799
3,17796,-1.0,luftdaten.info,Vinnytsia city,"Kyivs'ka St, 5",16.05.2021,-1,49.2380,28.4820
4,15952,337.0,Eco-City,Vinnytsia city,"3rd Chekhova Passage, 21",23.01.2021,-1,49.2268,28.5256


In [5]:
stations_about['id_station'] = "SaveEcoBot_" + stations_about['id_saveecobot'].astype('str')
stations_about['id_ecocity'] = stations_about['id_ecocity'].astype('int')
stations_about.loc[stations_about['network']=='Eco-City', 'id_station'] = "EcoCity_" + stations_about['id_ecocity'].astype('int').astype('str')
stations_about

,id_saveecobot,id_ecocity,network,locality,address,start_date,notes,lat,lng,id_station
0,14627,848,Eco-City,Vinnytsia city,"2nd Konstantynovycha Ln, 4",14.12.2020,-1,49.2307,28.4371,EcoCity_848
1,4152,767,Eco-City,Vinnytsia city,"Soborna St, 36",26.06.2020,Vinnytsia City Concil,49.2329,28.4704,EcoCity_767
2,1008,-1,SaveDnipro,Vinnytsia city,"Stetsenka St, 75",25.05.2019,-1,49.2393,28.4799,SaveEcoBot_1008
3,17796,-1,luftdaten.info,Vinnytsia city,"Kyivs'ka St, 5",16.05.2021,-1,49.2380,28.4820,SaveEcoBot_17796
4,15952,337,Eco-City,Vinnytsia city,"3rd Chekhova Passage, 21",23.01.2021,-1,49.2268,28.5256,EcoCity_337


In [6]:
stations_about = stations_about[['id_station', 'lat', 'lng']]
stations_about

,id_station,lat,lng
0,EcoCity_848,49.2307,28.4371
1,EcoCity_767,49.2329,28.4704
2,SaveEcoBot_1008,49.2393,28.4799
3,SaveEcoBot_17796,49.2380,28.4820
4,EcoCity_337,49.2268,28.5256


### 2.2 Download data from EcoCity<a class="anchor" id="2.2"></a>

[Back to Table of Contents](#0.1)

In [7]:
# Data from SaveEcoBot
ecocity_stations_about = pd.read_csv('../input/air-quality-monitoring-from-ecocity/ecocity_about_stations_2021.csv', header=0, sep=';')
ecocity_stations_about

,id_saveecobot,id_ecocity,network,locality,address,start_date,lat,lng,notes,source
0,15952.0,337,Eco-City,Vinnytsia city,"3rd Chekhova Passage, 21",23.01.2021,49.2267,28.5259,Oil and fat factory,https://eco-city.org.ua/?zoom=16&lat=49.226859...
1,4238.0,650,Eco-City,Vinnytsia region,Turbiv town,02.08.2020,49.3435,28.7298,NaN,https://eco-city.org.ua/?zoom=14&lat=49.343484...
2,4152.0,767,Eco-City,Vinnytsia city,"Soborna St, 36",26.06.2020,49.2329,28.4704,Vinnytsia City Concil,https://eco-city.org.ua/?zoom=16&lat=49.232859...
3,NaN,774,Eco-City,Vinnytsia city,"Vinnytsia, Zamostyanska Street, 7/35",NaN,49.2455,28.4936,Transparent office,https://eco-city.org.ua/?zoom=16&lat=49.245473...
4,NaN,790,Eco-City,Vinnytsia city,"Vinnytsia, Cosmonauts Avenue, 30",NaN,49.2268,28.4194,Transparent office,https://eco-city.org.ua/?zoom=15&lat=49.227009...
5,14627.0,848,Eco-City,Vinnytsia city,"2nd Konstantynovycha Ln, 4",14.12.2020,49.2307,28.4371,NaN,https://eco-city.org.ua/?zoom=17&lat=49.230728...
6,14855.0,1172,Eco-City,Vinnytsia region,Ivaniv village,28.12.2020,49.4838,28.3636,NaN,https://eco-city.org.ua/?zoom=11&lat=49.483674...
7,NaN,1315,Eco-City,Vinnytsia city,"Vinnytsia, str. Khmelnytsky shose, 95, VNTU, b...",16.11.2021,49.2330,28.4100,Vinnytsia National Technical University,https://eco-city.org.ua/?zoom=16&lat=49.232957...
8,NaN,1183,Eco-City,Vinnytsia city,"Vinnytsia, str. Tram, 3",NaN,49.2311,28.4371,Avalon,https://eco-city.org.ua/
9,NaN,761,Eco-City,Vinnytsia city,"Vinnytsia, 600th anniversary, 66V",NaN,49.2222,28.4242,Triumph,https://eco-city.org.ua/


In [8]:
ecocity_stations_about_city = ecocity_stations_about[ecocity_stations_about['locality']=='Vinnytsia city'].reset_index(drop=True)
ecocity_stations_about_city['id_ecocity'] = ecocity_stations_about_city['id_ecocity'].astype('int')
ecocity_stations_about_city

,id_saveecobot,id_ecocity,network,locality,address,start_date,lat,lng,notes,source
0,15952.0,337,Eco-City,Vinnytsia city,"3rd Chekhova Passage, 21",23.01.2021,49.2267,28.5259,Oil and fat factory,https://eco-city.org.ua/?zoom=16&lat=49.226859...
1,4152.0,767,Eco-City,Vinnytsia city,"Soborna St, 36",26.06.2020,49.2329,28.4704,Vinnytsia City Concil,https://eco-city.org.ua/?zoom=16&lat=49.232859...
2,NaN,774,Eco-City,Vinnytsia city,"Vinnytsia, Zamostyanska Street, 7/35",NaN,49.2455,28.4936,Transparent office,https://eco-city.org.ua/?zoom=16&lat=49.245473...
3,NaN,790,Eco-City,Vinnytsia city,"Vinnytsia, Cosmonauts Avenue, 30",NaN,49.2268,28.4194,Transparent office,https://eco-city.org.ua/?zoom=15&lat=49.227009...
4,14627.0,848,Eco-City,Vinnytsia city,"2nd Konstantynovycha Ln, 4",14.12.2020,49.2307,28.4371,NaN,https://eco-city.org.ua/?zoom=17&lat=49.230728...
5,NaN,1315,Eco-City,Vinnytsia city,"Vinnytsia, str. Khmelnytsky shose, 95, VNTU, b...",16.11.2021,49.2330,28.4100,Vinnytsia National Technical University,https://eco-city.org.ua/?zoom=16&lat=49.232957...
6,NaN,1183,Eco-City,Vinnytsia city,"Vinnytsia, str. Tram, 3",NaN,49.2311,28.4371,Avalon,https://eco-city.org.ua/
7,NaN,761,Eco-City,Vinnytsia city,"Vinnytsia, 600th anniversary, 66V",NaN,49.2222,28.4242,Triumph,https://eco-city.org.ua/


In [9]:
ecocity_stations_about_city['id_station'] = "EcoCity_" + ecocity_stations_about_city['id_ecocity'].astype('str')
ecocity_stations_about_city.loc[ecocity_stations_about_city['network']=='Eco-City', 'id_station'] = "EcoCity_" + ecocity_stations_about_city['id_ecocity'].astype('str')
ecocity_stations_about_city

,id_saveecobot,id_ecocity,network,locality,address,start_date,lat,lng,notes,source,id_station
0,15952.0,337,Eco-City,Vinnytsia city,"3rd Chekhova Passage, 21",23.01.2021,49.2267,28.5259,Oil and fat factory,https://eco-city.org.ua/?zoom=16&lat=49.226859...,EcoCity_337
1,4152.0,767,Eco-City,Vinnytsia city,"Soborna St, 36",26.06.2020,49.2329,28.4704,Vinnytsia City Concil,https://eco-city.org.ua/?zoom=16&lat=49.232859...,EcoCity_767
2,NaN,774,Eco-City,Vinnytsia city,"Vinnytsia, Zamostyanska Street, 7/35",NaN,49.2455,28.4936,Transparent office,https://eco-city.org.ua/?zoom=16&lat=49.245473...,EcoCity_774
3,NaN,790,Eco-City,Vinnytsia city,"Vinnytsia, Cosmonauts Avenue, 30",NaN,49.2268,28.4194,Transparent office,https://eco-city.org.ua/?zoom=15&lat=49.227009...,EcoCity_790
4,14627.0,848,Eco-City,Vinnytsia city,"2nd Konstantynovycha Ln, 4",14.12.2020,49.2307,28.4371,NaN,https://eco-city.org.ua/?zoom=17&lat=49.230728...,EcoCity_848
5,NaN,1315,Eco-City,Vinnytsia city,"Vinnytsia, str. Khmelnytsky shose, 95, VNTU, b...",16.11.2021,49.2330,28.4100,Vinnytsia National Technical University,https://eco-city.org.ua/?zoom=16&lat=49.232957...,EcoCity_1315
6,NaN,1183,Eco-City,Vinnytsia city,"Vinnytsia, str. Tram, 3",NaN,49.2311,28.4371,Avalon,https://eco-city.org.ua/,EcoCity_1183
7,NaN,761,Eco-City,Vinnytsia city,"Vinnytsia, 600th anniversary, 66V",NaN,49.2222,28.4242,Triumph,https://eco-city.org.ua/,EcoCity_761


In [10]:
ecocity_stations_about_city = ecocity_stations_about_city[['id_station', 'lat', 'lng']]
ecocity_stations_about_city

,id_station,lat,lng
0,EcoCity_337,49.2267,28.5259
1,EcoCity_767,49.2329,28.4704
2,EcoCity_774,49.2455,28.4936
3,EcoCity_790,49.2268,28.4194
4,EcoCity_848,49.2307,28.4371
5,EcoCity_1315,49.2330,28.4100
6,EcoCity_1183,49.2311,28.4371
7,EcoCity_761,49.2222,28.4242


### 2.3 Data aggregation<a class="anchor" id="2.3"></a>

[Back to Table of Contents](#0.1)

In [11]:
ecocity_stations_about_city_list = ecocity_stations_about_city['id_station'].unique().tolist()
ecocity_stations_about_city_list

['EcoCity_337',
 'EcoCity_767',
 'EcoCity_774',
 'EcoCity_790',
 'EcoCity_848',
 'EcoCity_1315',
 'EcoCity_1183',
 'EcoCity_761']

In [12]:
stations_about = stations_about[~stations_about['id_station'].isin(ecocity_stations_about_city_list)]
stations_about

,id_station,lat,lng
2,SaveEcoBot_1008,49.2393,28.4799
3,SaveEcoBot_17796,49.2380,28.4820


In [13]:
stations_about = pd.concat([ecocity_stations_about_city, stations_about], ignore_index=True)
stations_about

,id_station,lat,lng
0,EcoCity_337,49.2267,28.5259
1,EcoCity_767,49.2329,28.4704
2,EcoCity_774,49.2455,28.4936
3,EcoCity_790,49.2268,28.4194
4,EcoCity_848,49.2307,28.4371
5,EcoCity_1315,49.2330,28.4100
6,EcoCity_1183,49.2311,28.4371
7,EcoCity_761,49.2222,28.4242
8,SaveEcoBot_1008,49.2393,28.4799
9,SaveEcoBot_17796,49.2380,28.4820


In [14]:
# Add data about stations of the Center for Hydrometeorology in Vinnytsia region (http://meteo.vn.ua/api/api.php)
# Source: https://www.vmr.gov.ua/Branches/Lists/Ecology/ShowContent.aspx?ID=24
num = len(stations_about)
stations_about.loc[num, 'id_station'] = 'VinnCHM_1'
stations_about.loc[num, 'lat'] = 49.241962 
stations_about.loc[num, 'lng'] = 28.478482
stations_about.loc[num+1, 'id_station'] = 'VinnCHM_2'
stations_about.loc[num+1, 'lat'] = 49.231392
stations_about.loc[num+1, 'lng'] = 28.518974
stations_about

,id_station,lat,lng
0,EcoCity_337,49.226700,28.525900
1,EcoCity_767,49.232900,28.470400
2,EcoCity_774,49.245500,28.493600
3,EcoCity_790,49.226800,28.419400
4,EcoCity_848,49.230700,28.437100
5,EcoCity_1315,49.233000,28.410000
6,EcoCity_1183,49.231100,28.437100
7,EcoCity_761,49.222200,28.424200
8,SaveEcoBot_1008,49.239300,28.479900
9,SaveEcoBot_17796,49.238000,28.482000


## 3. Prediction<a class="anchor" id="3"></a>

[Back to Table of Contents](#0.1)

In [15]:
stations_about_list = stations_about['id_station'].tolist()
stations_about_list

['EcoCity_337',
 'EcoCity_767',
 'EcoCity_774',
 'EcoCity_790',
 'EcoCity_848',
 'EcoCity_1315',
 'EcoCity_1183',
 'EcoCity_761',
 'SaveEcoBot_1008',
 'SaveEcoBot_17796',
 'VinnCHM_1',
 'VinnCHM_2']

In [16]:
st_par = pd.read_csv('../input/air-quality-monitoring/stations_parameters_prediction.csv', header=0, sep=';')
st_par

,id_station,PM2.5_k1,PM2.5_k0,PM2.5_a1,PM2.5_b1,PM10_k1,PM10_k0,PM10_a1,PM10_b1
0,EcoCity_337,1,1,1,1,1,1,1,1
1,EcoCity_767,1,1,1,1,1,1,1,1
2,EcoCity_774,1,1,1,1,1,1,1,1
3,EcoCity_790,1,1,1,1,1,1,1,1
4,EcoCity_848,1,1,1,1,1,1,1,1
5,EcoCity_1315,1,1,1,1,1,1,1,1
6,EcoCity_1183,1,1,1,1,1,1,1,1
7,EcoCity_761,1,1,1,1,1,1,1,1
8,SaveEcoBot_1008,1,1,1,1,1,1,1,1
9,SaveEcoBot_17796,1,1,1,1,1,1,1,1


In [17]:
def f(st_par, id_station, indicator_name, t):
    # Prediction value of th indicator_name for the station with id_station 
    # in the time moment t by parameters from the st_par
    
    df = st_par[st_par['id_station']==id_station].reset_index(drop=True)
    k1 = float(df.loc[0, indicator_name+"_k1"])
    k0 = float(df.loc[0, indicator_name+"_k0"])
    a1 = float(df.loc[0, indicator_name+"_a1"])
    b1 = float(df.loc[0, indicator_name+"_b1"])
        
    return k0 + k1*t + a1*np.sin(t) + b1*np.cos(t)

In [18]:
res = pd.DataFrame(columns=['id_station', 't', 'y'])
res.loc[0,'t']=1

In [19]:
res = pd.DataFrame(columns=['id_station', 'indicator_name', 't', 'y'])
w = 0
for id_station in stations_about_list:
    for indicator_name in ['PM2.5', 'PM10']:
        for t in range(10):
            res.loc[w, 'id_station'] = id_station
            res.loc[w, 'indicator_name'] = indicator_name
            res.loc[w, 't'] = t
            res.loc[w, 'y'] = f(st_par, id_station, indicator_name, t) + np.random.normal(0, .1, 1)[0]
            w += 1

In [20]:
res

,id_station,indicator_name,t,y
0,EcoCity_337,PM2.5,0,1.984141
1,EcoCity_337,PM2.5,1,3.450029
2,EcoCity_337,PM2.5,2,3.560624
3,EcoCity_337,PM2.5,3,3.107028
4,EcoCity_337,PM2.5,4,3.527677
...,...,...,...,...
235,VinnCHM_2,PM10,5,5.184887
236,VinnCHM_2,PM10,6,7.761574
237,VinnCHM_2,PM10,7,9.343649
238,VinnCHM_2,PM10,8,9.797977


In [21]:
stations_about

,id_station,lat,lng
0,EcoCity_337,49.226700,28.525900
1,EcoCity_767,49.232900,28.470400
2,EcoCity_774,49.245500,28.493600
3,EcoCity_790,49.226800,28.419400
4,EcoCity_848,49.230700,28.437100
5,EcoCity_1315,49.233000,28.410000
6,EcoCity_1183,49.231100,28.437100
7,EcoCity_761,49.222200,28.424200
8,SaveEcoBot_1008,49.239300,28.479900
9,SaveEcoBot_17796,49.238000,28.482000


## 4. Mapping<a class="anchor" id="4"></a>

[Back to Table of Contents](#0.1)

In [22]:
m = folium.Map(location=[49.2355, 28.474], zoom_start=13, tiles="Stamen Terrain")

for i in range(len(stations_about)):
    marker = folium.Marker(
        location=[float(stations_about.loc[i, 'lat']), float(stations_about.loc[i, 'lng'])],
        popup=folium.Popup(max_width=450)
    )
    marker.add_to(m)

m

In [23]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
data = json.loads(requests.get(f"{url}/vis1.json").text)

m = folium.Map(location=[49.2355, 28.474], zoom_start=13)

for i in range(len(stations_about)):
    marker = folium.Marker(
        location=[float(stations_about.loc[i, 'lat']), float(stations_about.loc[i, 'lng'])],
        popup=folium.Popup(max_width=450).add_child(
            folium.Vega(data, width=450, height=450)   # Still need to replace data to res for given id_station and indicator 
        ),
    )
    marker.add_to(m)

m

I hope you find this notebook useful and enjoyable.

Your comments and feedback are most welcome.

[Go to Top](#0)